In [0]:
# uninstall older version and install tensorflow 2.0
!pip uninstall tensorflow
!pip install tensorflow==2.0.0-beta1

Please restart the colab so as to make the tf 2.0 be effective.

In [105]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


In [0]:
import urllib
from os import listdir
from os.path import isdir
import shutil
from PIL import Image
import re
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

## Step 0: Create Dataset

Here we downloaded our original dataset from [head pose image database](http://www-prima.inrialpes.fr/perso/Gourier/Faces/HPDatabase.html). And only four directions (up, down, left and right) are used to form the dataset of this demo. 

In [3]:
dataset = "http://www-prima.inrialpes.fr/perso/Gourier/Faces/HeadPoseImageDatabase.tar.gz"
urllib.request.urlretrieve(dataset,'data.tar.gz') 

('data.tar.gz', <http.client.HTTPMessage at 0x7f37f3eabb00>)

In [0]:
# tar file
!mkdir data
!mkdir headpose
!tar -C ./data -zxvf data.tar.gz

In [0]:
# # Version 1: create dataset in four directions
# def createDataset(input_path, output_path):
#   personfolders = listdir(input_path)
#   for person in personfolders:
#     path = input_path + person
#     if isdir(path):
#       imagesList = listdir(path)
#       for image in imagesList:
#         image_path = path + '/' + image
#         if image[::-1][:9] == '-90+0.jpg'[::-1]:
#             shutil.copy(image_path, output_path + person + 'down.jpg')
#         elif image[::-1][:9] == '+0-90.jpg'[::-1]:
#             shutil.copy(image_path, output_path + person + 'right.jpg')
#         elif image[::-1][:9] == '+0+90.jpg'[::-1]:
#             shutil.copy(image_path, output_path + person + 'left.jpg')
#         elif image[::-1][:9] == '+90+0.jpg'[::-1]:
#             shutil.copy(image_path, output_path + person + 'up.jpg')

In [0]:
def createDataset(input_path, output_path):
  personfolders = listdir(input_path)
  for person in personfolders:
    path = input_path + person
    if isdir(path):
      imagesList = listdir(path)
      for image in imagesList:
        image_path = path + '/' + image
        if image[::-1][:4] == '.jpg'[::-1]:
            image_name = image[::-1][:10][::-1]
            tilt = int(re.findall('[+-]{1}[0-9]*',image_name)[0])
            pan = int(re.findall('[+-]{1}[0-9]*',image_name)[1])
            output_name = output_path + image[::-1][4:][::-1]
            if person == 'Front':
                shutil.copy(image_path, output_name + 'front.jpg')
            else:
                if abs(tilt) <= 15 and abs(pan) <= 15:
                    shutil.copy(image_path, output_name + 'front.jpg')
                elif tilt <= -30 and abs(pan) <= 45:
                    shutil.copy(image_path, output_name + 'down.jpg')
                elif tilt >= 30 and abs(pan) <= 45:
                    shutil.copy(image_path, output_name + 'up.jpg')   
                elif (abs(tilt) > 30 and pan < -45) or (abs(tilt) <= 15 and pan < -15):
                    shutil.copy(image_path, output_name + 'left.jpg') 
                elif (abs(tilt) > 30 and pan > 45) or (abs(tilt) <= 15 and pan > 15):
                    shutil.copy(image_path, output_name + 'right.jpg')

In [0]:
createDataset('./data/','./headpose/')

## Step 1: Load Images and Pre-process

### (a) load images

In [0]:
def name2label(name):
  label_dict = {'front':0, 'up':1, 'down':2, 'left': 3, 'right': 4}
  direction = re.findall('[0-9]{1}[a-z]+',name)[0][1:]
  return label_dict[direction]

def loadImages(path, imgH, imgW):
  imagesList = listdir(path)
  loadedImages = []
  labels = []
  for image in imagesList:
    img = Image.open(path + image)
    img = img.resize((width, height))
    loadedImages.append(img)
    img_label = name2label(image)
    labels.append(img_label) 
  return loadedImages, labels

In [0]:
height = 224
width = 224

images, labels = loadImages('./headpose/', height, width)

In [0]:
# # version 1: process images without data augmentation
# def preprocess_images(imagesList):
#   n =  len(imagesList)
#   images_array = np.zeros(shape=(n, width, height, 3), dtype='float32')
#   for i in range(n):
#     images_array[i,:,:,:] = np.asarray(imagesList[i], dtype='float32')
#   images_array = images_array / 127.5 - 1  #refer keras image preprocessing utils(tf mode)
#   return images_array

In [0]:
# images_array = preprocess_images(images_array)
# labels_array = np.array(labels_array)

### (b)data augmentation

In [0]:
def data_aug(x):
  x = np.expand_dims(x,0)
  # left-translation with 50px
  x1 = tf.image.pad_to_bounding_box(x, 0, 10, 224, 234)
  x1 = tf.image.crop_to_bounding_box(x1, 0, 0, 224, 224).numpy()[0]
  
  # top-translation with 50px
  x2 = tf.image.pad_to_bounding_box(x, 10, 0, 234, 224)
  x2 = tf.image.crop_to_bounding_box(x2, 0, 0, 224, 224).numpy()[0]
  
  # right-translation with 50px
  x3 = tf.image.pad_to_bounding_box(x, 0, 0, 224, 234)
  x3 = tf.image.crop_to_bounding_box(x3, 0, 10, 224, 224).numpy()[0]
  
  # bottom-translation with 50px
  x4 = tf.image.pad_to_bounding_box(x, 0, 0, 234, 224)
  x4 = tf.image.crop_to_bounding_box(x4, 10, 0, 224, 224).numpy()[0]
  
  # add noise
  noise = tf.random.normal(shape=tf.shape(x), mean=0.0, stddev=0.25,dtype=tf.float32)
  x5 = tf.add(x, noise).numpy()[0]
                           
  
  return [x[0],x1,x2,x3,x4,x5]
                           

### (c) preprocessing

In [0]:
def preprocess_images(imagesList, labels):
  n =  len(images)
  images_array = []
  labels_aug = []
  for i in range(n):
    thres = np.random.uniform(size=1)
    if labels[i] == 0 and thres < 0.7:
      images_aug = data_aug(np.asarray(imagesList[i], dtype='float32'))
      images_array += images_aug
      labels_aug += [labels[i]]*6
    elif labels[i] == 1 and thres < 0.5:
      images_aug = data_aug(np.asarray(imagesList[i], dtype='float32'))
      images_array += images_aug
      labels_aug += [labels[i]]*6
    elif labels[i] == 2 and thres < 0.5:
      images_aug = data_aug(np.asarray(imagesList[i], dtype='float32'))
      images_array += images_aug
      labels_aug += [labels[i]]*6
    elif labels[i] == 3 and thres < 0.3:
      images_aug = data_aug(np.asarray(imagesList[i], dtype='float32'))
      images_array += images_aug
      labels_aug += [labels[i]]*6
    elif labels[i] == 4 and thres < 0.3:
      images_aug = data_aug(np.asarray(imagesList[i], dtype='float32'))
      images_array += images_aug  
      labels_aug += [labels[i]]*6
  images_array = np.asarray(images_array,dtype='float32') / 127.5 - 1  #refer keras image preprocessing utils(tf mode)
  return images_array, labels_aug

In [306]:
np.random.seed(123)
images_array, labels_array = preprocess_images(images, labels)
labels_array = np.array(labels_array)
print(images_array.shape)

(6348, 224, 224, 3)


In [307]:
np.unique(labels_array,return_counts=True)

(array([0, 1, 2, 3, 4]), array([1320, 1314, 1290, 1188, 1236]))

### (d) shuffle dataset

In [0]:
images_array, labels_array = shuffle(images_array, labels_array, random_state=42)

### (e) split training and test dataset

In [0]:
# images_train, images_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.33, random_state=1234)

## Step 2: Train Model

In [0]:
mobile_net = tf.keras.applications.MobileNet(input_shape=(224, 224, 3), include_top=False)
mobile_net.trainable=False  #non-trainable

In [0]:
mobile_net.summary()

In [0]:
features = mobile_net.predict(images_array)

In [0]:
# build a model based on mobile_net
snake_model= tf.keras.Sequential([ 
#   tf.keras.layers.Dense(1000,activation='relu',input_shape=(7,7,1024,)),  
#   tf.keras.layers.Dense(512,activation='relu',input_shape=(7,7,1024,)),
#   tf.keras.layers.Dense(128,activation='relu',input_shape=(7,7,1024,)),  
#   tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Flatten(input_shape=(7,7,1024,)),
  tf.keras.layers.Dense(1024,activation='relu'),
  tf.keras.layers.Dense(512,activation='relu'),
  tf.keras.layers.Dense(100,activation='relu'),
  tf.keras.layers.Dense(5,activation='softmax')])

In [0]:
snake_model.compile(optimizer=tf.optimizers.Adam(1e-4), 
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=["accuracy"])
history=snake_model.fit(features,labels_array,batch_size=256,epochs=20,validation_split=0.2)

In [183]:
pred = snake_model.predict(features)
pred_y = np.argmax(pred, axis=1)
np.unique(pred_y != labels_array, return_counts=True)

(array([False,  True]), array([5763,  915]))

In [0]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(history.epoch,history.history['loss'],label='train')
plt.plot(history.epoch,history.history['val_loss'],'--',label='validation')
plt.subplot(122)
plt.plot(history.epoch,history.history['accuracy'],label='train')
plt.plot(history.epoch,history.history['val_accuracy'],'--',label='validation')
plt.legend()
plt.show()

In [328]:
# img_test = Image.open('./me/front.jpg')
# img_test = img_test.resize((width, height))
# img_test = np.asarray(img_test, dtype='float32')
# img_test = img_test / 127.5 - 1
# img_test = np.expand_dims(img_test,0)
# temp = mobile_net.predict(img_test)
# snake_model.predict(temp)

array([[1.5718588e-01, 8.3859557e-01, 5.2573945e-04, 3.3155931e-04,
        3.3612815e-03]], dtype=float32)

### Step 3: Save Model

In [0]:
!mkdir model
snake_model.save('./model/model.h5')

In [0]:
!sudo pip install tensorflowjs

In [0]:
!mkdir tfjs_model
!tensorflowjs_converter --input_format=keras ./model/model.h5 ./tfjs_model

In [321]:
!zip -r model.zip ./tfjs_model/

  adding: tfjs_model/ (stored 0%)
  adding: tfjs_model/group1-shard50of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard48of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard6of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard39of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard29of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard41of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard24of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard46of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard13of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard26of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard15of50.bin (deflated 8%)
  adding: tfjs_model/model.json (deflated 81%)
  adding: tfjs_model/group1-shard45of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard4of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard25of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard49of50.bin (deflated 8%)
  adding: tfjs_model/group1-shard19of50.bin (deflat

In [0]:
!rm -rf tfjs_model
!rm -rf model
!rm -rf model.zip